# Capstone Project - Predicting Car Accident Severity in Seattle
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem <a name="introduction"></a>

In this project i will try to predict the severity of a car in Seattle crash using a collision dataset. This report will be targeted to stakeholders intreseted in predicting the outcome of a car crash, for example The Seattle Polcie Department.

The severiy of a car crash can split into two groups - Property Damage only and Injury Collission. These are idetfied as 1 and 2 respectevly in the dataset.

i will use my data science knowldge that Ive gained for mthe course and outside matreials to make a Machine Learning model wit haccuracy high enough to be useful for the stakeholders.

## Data

In [44]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')




In [38]:
#Import dataset
df = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv")

#Set OBJECTID as Index for convienience
df.set_index("OBJECTID");


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Based on my inution the information that is knonwn when an a collison is reported are:

* ADDRTYPE - **Collision location**

* COLLISIONTYPE - **The type of Collision**
* PERSONCOUNT - **The toal number of people involved in the collision**
* VEHCOUNT - **The number of veicedls involved i nthe incident**
* JUNCTIONTYPE - **Category of junction at which collision took place**
* WEATHER - **A description of the weather conditions during the time of the collision**
* ROADCOND - **The condition of the road during the collision**
* LIGHTCOND - **




In [39]:
columns_new = ['SEVERITYCODE', 'ADDRTYPE', 'COLLISIONTYPE','PERSONCOUNT', 
               'VEHCOUNT', 'JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'HITPARKEDCAR']

df_new = df[columns_new]

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new.fillna(df_new.mean(), inplace=True)

In [62]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

df_new.apply(LabelEncoder().fit_transform)

sns.pairplot(X_train)
plt.show()

TypeError: ignored

In [47]:
X_train.isnull().sum()

PERSONCOUNT                                                       0
VEHCOUNT                                                          0
ADDRTYPE_Alley                                                    0
ADDRTYPE_Block                                                    0
ADDRTYPE_Intersection                                             0
COLLISIONTYPE_Angles                                              0
COLLISIONTYPE_Cycles                                              0
COLLISIONTYPE_Head On                                             0
COLLISIONTYPE_Left Turn                                           0
COLLISIONTYPE_Other                                               0
COLLISIONTYPE_Parked Car                                          0
COLLISIONTYPE_Pedestrian                                          0
COLLISIONTYPE_Rear Ended                                          0
COLLISIONTYPE_Right Turn                                          0
COLLISIONTYPE_Sideswipe                         

In [40]:
df_new.dropna(axis=0, subset=['SEVERITYCODE'], inplace=True)
y = df_new.SEVERITYCODE
df_new.drop(['SEVERITYCODE'], axis=1, inplace=True)

dataTypeSeries = df_new.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries);

Data type of each column of Dataframe :
ADDRTYPE         object
COLLISIONTYPE    object
PERSONCOUNT       int64
VEHCOUNT          int64
JUNCTIONTYPE     object
WEATHER          object
ROADCOND         object
LIGHTCOND        object
HITPARKEDCAR     object
dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_new, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [57]:
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()



# One-hot encode the data (to shorten the code, we use pandas)
#X_train = pd.get_dummies(X_train_full)
#X_valid = pd.get_dummies(X_valid_full)
#X_train, X_valid = X_train.align(X_valid, join='left', axis=1)

In [52]:

model = XGBRegressor(early_stopping_rounds=5)
n_estimators = [100, 200, 300, 400, 500]
learning_rate = [0.0001, 0.001, 0.005, 0.01, 0.1]
max_depth = [3, 5, 7, 9]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot results
scores = np.array(means).reshape(len(learning_rate), len(n_estimators))
for i, value in enumerate(learning_rate):
    plt.plot(n_estimators, scores[i], label='learning_rate: ' + str(value))
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Log Loss')
plt.savefig('n_estimators_vs_learning_rate.png')
plt.show()

KeyboardInterrupt: ignored

In [51]:
# Calculate MAE
model2 = XGBClassifier(learning_rate=0.1, n_estimators=300)
model2.fit(X_train, y_train)
predictions_1 = np.round(model2.predict(X_valid))
mae_1 = mean_absolute_error(predictions_1, y_valid)

print("Mean Absolute Error:" , mae_1)

#sklearn.metrics.r2_score(y_true, y_pred)

Mean Absolute Error: 0.24307178631051754


In [46]:
df_new.isnull().sum()

ADDRTYPE         1926
COLLISIONTYPE    4904
PERSONCOUNT         0
VEHCOUNT            0
JUNCTIONTYPE     6329
WEATHER          5081
ROADCOND         5012
LIGHTCOND        5170
HITPARKEDCAR        0
dtype: int64

In [ ]:
0.31942913144945534
0.24248105817387955